In [1]:
import os

In [2]:
%pip install langchain-groq langchain-core python-dotenv PyPDF2 pandas

In [3]:
import sys
print(sys.executable)

In [4]:
! "d:\MCQs_Generator\env\python.exe" -m pip install langchain-groq

In [5]:
! "d:\MCQs_Generator\env\python.exe" -m pip install langchain-community

In [6]:
# from langchain.llms import OpenAI  <-- (Old OpenAI Import)
# from langchain.chat_models import ChatOpenAI <-- (Old OpenAI Import)
from langchain_groq import ChatGroq # <--- (New Groq Import)

# Updated imports for langchain v0.2+
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableSequence

# from langchain.callbacks import get_openai_callback <-- (This specific callback is for OpenAI only. You can remove it or comment it out.)

import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [7]:
load_dotenv()

In [8]:
key = os.getenv("GROQ_API_KEY")

In [9]:
llm = ChatGroq(
    groq_api_key = key, # Ensure this is in your .env file
    model_name="llama-3.3-70b-versatile"  # Updated to currently supported Groq model
)

In [10]:
llm

In [12]:
with open("D:\\MCQs_Generator\\Response.json", "r") as f:
    RESPONSE_JSON = json.load(f)

In [13]:
print(RESPONSE_JSON)

In [14]:
TEMPLATE = """
Text:{text}
You are an expert an MCQ maker. Given the above text, it is your job to create 
a quiz of {number} multiple choice questions for {subject} students in {tone}.
Make sure the questions are not repeated and check all the questions to be confirming.
Make sure to format your response like RESPONSE_JSON below and use it as a guide
Ensure to make {number} MCQs.
### RESPONSE_JSON
{RESPONSE_JSON}
"""

In [15]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],
    template=TEMPLATE
    )

In [16]:
# Using modern LCEL (LangChain Expression Language) instead of deprecated LLMChain
quiz_chain = quiz_generation_prompt | llm | StrOutputParser()

In [17]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz
for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at maz 50 words for complexity analysis.
If the quiz is not as per with the cognitive and analytical capabilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student ability.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [18]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
    )

In [19]:
review_chain = quiz_evaluation_prompt | llm | StrOutputParser()

In [20]:
# Using LCEL to create a combined chain that passes outputs between chains
# RunnablePassthrough allows passing the original inputs along with new outputs
from langchain_core.runnables import RunnableLambda

def create_review_input(inputs):
    """Prepare inputs for review chain by adding quiz output"""
    return {
        **inputs,
        "quiz": inputs.get("quiz", "")
    }

# Combined chain: first generates quiz, then reviews it
generate_evaluate_chain = (
    RunnablePassthrough.assign(quiz=quiz_chain)
    | RunnableLambda(lambda x: {**x, "quiz": x["quiz"]})
    | RunnablePassthrough.assign(review=review_chain)
)

In [21]:
PATH = "D:\\MCQs_Generator\\data.txt"

In [22]:
with open(PATH, "r") as file:
    TEXT = file.read()

In [23]:
print(TEXT)

In [24]:
#MCQ KO GENERATE KRNA HAI

In [25]:
RESPONSE_JSON

In [26]:
import json
json.dumps(RESPONSE_JSON)

In [27]:
response = generate_evaluate_chain.invoke({
    "text": TEXT,
    "number": 5,
    "subject": "AI",
    "tone": "Simple",
    "RESPONSE_JSON": json.dumps(RESPONSE_JSON)
})
print(response)

In [28]:
response

In [30]:
quiz = response.get("quiz")

In [31]:
json.loads(quiz)

In [32]:
quiz = json.loads(quiz)

In [33]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({
        "MCQ": mcq,
        "Choices": options,
        "Correct": correct
    })

In [34]:
quiz_table_data

In [37]:
df = pd.DataFrame(quiz_table_data)
df

In [39]:
df.to_csv("D:\\MCQs_Generator\\quiz.xlsx", index=False)